In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [66]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train[0:500]
y_train = y_train[0:500]

In [67]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [68]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [69]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
500/500 [==============================] - 0s 165us/sample - loss: 1.7158 - acc: 0.5100
Epoch 2/10
500/500 [==============================] - 0s 133us/sample - loss: 0.7162 - acc: 0.8380
Epoch 3/10
500/500 [==============================] - 0s 146us/sample - loss: 0.4169 - acc: 0.8920
Epoch 4/10
500/500 [==============================] - 0s 115us/sample - loss: 0.2888 - acc: 0.9240
Epoch 5/10
500/500 [==============================] - 0s 129us/sample - loss: 0.2148 - acc: 0.9560
Epoch 6/10
500/500 [==============================] - 0s 134us/sample - loss: 0.1578 - acc: 0.9720
Epoch 7/10
500/500 [==============================] - 0s 139us/sample - loss: 0.1250 - acc: 0.9800
Epoch 8/10
500/500 [==============================] - 0s 131us/sample - loss: 0.0940 - acc: 0.9940
Epoch 9/10
500/500 [==============================] - 0s 118us/sample - loss: 0.0742 - acc: 0.9960
Epoch 10/10
500/500 [==============================] - 0s 135us/sample - loss: 0.0563 - acc: 1.0000


In [70]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 51us/sample - loss: 0.4957 - acc: 0.8483


[0.49566403232812883, 0.8483]

In [71]:
from sklearn.neural_network import MLPClassifier
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))

In [72]:
%%time
clf = MLPClassifier(hidden_layer_sizes=(32,),learning_rate='adaptive', max_iter=1000)
clf.fit(X_train_flat, y_train)
print('Train acc: %.4f, Test acc: %.4f' % (clf.score(X_train_flat, y_train), clf.score(X_test_flat, y_test)))

Train acc: 1.0000, Test acc: 0.8367
CPU times: user 1.41 s, sys: 340 ms, total: 1.75 s
Wall time: 880 ms


In [73]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [64]:
%%time
kernel = RBF()
train_size = 500
gpc = GaussianProcessClassifier(kernel, random_state=23, n_restarts_optimizer=5)
gpc.fit(X_train_flat[0:train_size], y_train[0:train_size])
print('kernel hyperparameters:%s, logevidence: %.2f ' % (gpc.kernel_, gpc.log_marginal_likelihood_value_))

kernel hyperparameters:CompoundKernel(2.08, 2, 2.03, 2.06, 2.08, 2.11, 2.08, 2.08, 2.14, 2.08), logevidence: -120.46 
CPU times: user 1min 10s, sys: 19.9 s, total: 1min 30s
Wall time: 45.2 s


In [74]:
folds = 5
batch= 1000
for i in range(folds):
    print(gpc.score(X_test_flat[i*batch:i*batch+batch], y_test[i*batch:i*batch+batch]))

0.72
0.688
0.702
0.706
0.704


In [79]:
correct = np.equal(gpc.predict(X_test_flat[0:1000]), y_test[0:1000])

In [84]:
y_test[0:1000][correct == False]

array([5, 9, 6, 5, 6, 2, 2, 5, 5, 5, 6, 5, 8, 3, 4, 6, 9, 2, 7, 8, 9, 7,
       6, 9, 7, 4, 4, 2, 5, 4, 7, 5, 5, 2, 5, 6, 5, 4, 4, 2, 7, 8, 8, 5,
       3, 4, 6, 5, 3, 6, 5, 3, 8, 7, 5, 9, 7, 2, 3, 4, 2, 4, 5, 2, 8, 6,
       5, 8, 8, 7, 8, 4, 4, 4, 7, 9, 6, 5, 2, 5, 9, 2, 4, 2, 5, 8, 5, 6,
       8, 5, 3, 5, 5, 5, 7, 2, 2, 5, 5, 7, 8, 8, 4, 3, 5, 3, 8, 2, 5, 7,
       5, 8, 5, 0, 2, 6, 2, 2, 6, 8, 5, 6, 3, 7, 5, 5, 9, 5, 8, 5, 8, 4,
       6, 9, 3, 5, 4, 2, 6, 3, 4, 8, 8, 3, 4, 2, 8, 5, 8, 8, 2, 7, 7, 0,
       8, 6, 4, 6, 8, 3, 8, 5, 4, 8, 8, 6, 8, 5, 7, 8, 2, 2, 3, 5, 8, 2,
       9, 3, 2, 5, 8, 5, 2, 2, 5, 7, 2, 0, 6, 7, 9, 9, 5, 7, 7, 8, 5, 4,
       5, 4, 8, 0, 5, 2, 3, 6, 2, 4, 2, 4, 5, 4, 5, 7, 4, 5, 8, 5, 8, 5,
       8, 3, 5, 8, 8, 5, 2, 4, 2, 2, 7, 6, 2, 8, 8, 5, 5, 5, 2, 8, 8, 8,
       8, 4, 9, 2, 5, 7, 8, 4, 9, 8, 2, 2, 2, 2, 7, 4, 5, 5, 8, 6, 2, 6,
       3, 7, 8, 7, 5, 6, 5, 4, 7, 6, 4, 5, 6, 2, 5, 8], dtype=uint8)

In [86]:
probas = gpc.predict_proba(X_test_flat[0:1000])

In [87]:
probas.shape

(1000, 10)